In [3]:
import os
import logging
import torch
from transformers import GenerationConfig, AutoModelForQuestionAnswering, AutoTokenizer, pipeline, AutoModelForCausalLM
# 추가학습한 koalpaca inference
device = f"cuda:0" if torch.cuda.is_available() else "cpu"
completion_model_name = '/hdd/hkyoon95/polyglot-5.8b-koalpaca-emotion'
# completion_model_name = 'beomi/KoAlpaca-KoRWKV-6B'
generation_config = GenerationConfig.from_pretrained(completion_model_name)
tokenizer = AutoTokenizer.from_pretrained(completion_model_name)
model = AutoModelForCausalLM.from_pretrained(
    completion_model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=device, non_blocking=True)

/home/hkyoon95/anaconda3/envs/koalpaca/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2023-08-17 02:47:39,662] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Loading checkpoint shards: 100%|██████████| 3/3 [00:20<00:00,  6.92s/it]


In [30]:
def gen(prompt=None, max_new_tokens=128, temperature=0.3):
    # max_new_tokens: generate token 제한 개수
    # temperature: sampling method hyperparameter 높을수록 random하게 generate할 확률이 증가함
    user_input = []
    output = []
    for i in range(3):
        if i == 0:
            text = input("Please enter something: ")
            x = "### 질문: {}\n\n### 답변:".format(text)
            user_input.append(text)
        else:
            text = input("Please enter something: ")
            user_input.append(text)
            x = x + gen_text[0].replace(x, '').split('###')[0].strip('\n').strip(' ') \
                    + '\n\n### 질문: {}\n\n### 답변: '.format(text)
        input_ids = tokenizer.encode(x, return_tensors="pt").to('cuda:0')
        gen_tokens = model.generate(
            input_ids, 
            max_new_tokens=max_new_tokens,
            num_return_sequences=1, 
            temperature=temperature,
            no_repeat_ngram_size=6,
            do_sample=True,
        )
        gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
        # return gen_text
        print('입력: ', text)
        print('출력: ', gen_text[0].replace(x, '').split('###')[0])
        output.append(gen_text[0].replace(x, '').split('###')[0].strip('\n').strip(' '))
        
    return user_input, output
user_input, output = gen(temperature=0.6)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


입력:  오늘 회사에서 일을 망쳤어
출력:   회사에서 일을 망치셨군요. 어떤 상황이었나요?


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


입력:  발표를 너무 못했어
출력:   발표를 잘 못 하셔서 속상하셨겠어요.


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


입력:  같이 준비한 동료들한테 미안해
출력:   준비한 동료들에게 미안한 마음이 크시군요.


In [ ]:
for i in range(len(user_input)):
    print('입력: {}'.format(user_input[i]))
    print('출력: {}'.format(output[i]))

In [ ]:
# test original koalpaca
from transformers import pipeline

pipe = pipeline('text-generation', model='beomi/KoAlpaca-KoRWKV-6B', device=0)

In [ ]:
def gen(x, temp):
    a = pipe(
        f'### 질문: {x}\n\n### 답변:', 
        max_new_tokens=1024, 
        return_full_text=False, 
        do_sample=True,
        # top_p=0.9,
        temperature=temp,
        early_stopping=True,
        repetition_penalty=1.2,
    )
    print(a[0]['generated_text'])
gen('삼촌이 추석 때 취업하라 그래서 짜증나', temp=0.3)

In [32]:
prompt = '오늘 점심 뭐 먹을까?'
generated_text = gen(user_input = prompt, prompt = '다음 문장에 공감 또는 대답해주세요', temperature=0.3)
print(generated_text.split('###')[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



같이 먹을래?


In [13]:
PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n### Input(입력):오늘 점심 뭐 먹을까?\n### Response(응답):오늘은 짬뽕이 어떠신가요.\n### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
    ),
}

def gen(user_input=None, prompt, max_new_tokens=128, temperature=0.3):
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)
    
    input_ids = tokenizer.encode(x, return_tensors="pt").to('cuda:0')
    gen_tokens = model.generate(
        input_ids, 
        max_new_tokens=max_new_tokens, 
        num_return_sequences=1, 
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,
    )
    gen_text = tokenizer.batch_decode(gen_tokens, skip_special_tokens=True)
    # return gen_text
    return gen_text[0].replace(x, '')
prompt = '좋은 거 같아'
generated_text = gen(user_input = prompt, prompt = '', temperature=1.1)
print(generated_text.split('###')[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


 좋은 선택이에요.


In [ ]:
prompt = "추석에 삼촌이 와서 취업 했냐고 물어보면 진짜 짜증나"
for i in range(10):
    generated_text = gen(user_input = '내 말에 공감해줘', prompt = prompt, temperature=0.1)
    print(generated_text.split('###')[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



"추석 때 취업 관련한 언급이 불편하게 느껴질 수 있다는 것에 공감합니다."


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



"추석 때 취업 관련한 언급이 불편하게 느껴질 수 있겠네요. 스트레스 받는 상황이겠지만, 삼촌과의 대화에서 더 나은 대응을 할 수 있는 방법을 모색해보는 것이 좋겠습니다."


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


...


"추석 때 취업 관련한 언급이 불편하게 느껴질 수 있겠네요. 스트레스 받는 상황이겠지만, 삼촌과의 대화에서 더 나은 대응을 할 수 있는 방법을 모색해보는 것이 좋겠습니다."


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



"추석 때 취업 관련한 언급이 불편하게 느껴질 수 있겠네요. 스트레스 받는 상황이겠지만, 삼촌과의 대화에서 더 나은 대응을 할 수 있는 방법을 모색해보는 것이 좋겠습니다."


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.



"추석 때 취업 관련한 언급이 불편하게 느껴질 수 있다는 것에 공감합니다."

"추석 때 취업 관련한 언급이 불편하게 느껴질 수 있겠네요. 스트레스 받지 마시고, 즐거운 추석 연휴 보내시길 바랍니다."
